In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from itertools import combinations
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# CSV file
file_path = 'data/iith_foml_2023_train.csv'
data = pd.read_csv(file_path)

In [3]:
# Assuming the target variable column is named "Target Variable (Discrete)"
X = data.drop(columns=['Target Variable (Discrete)','Feature 21 (Discrete)','Feature 22 (Discrete)','Feature 23 (Discrete)','Feature 24'])
y = data['Target Variable (Discrete)']

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=7)


In [5]:
lda.fit(X_imputed, y)

LinearDiscriminantAnalysis(n_components=7)

In [6]:
X_imputed = lda.transform(X_imputed)

In [7]:
rf_model = KNeighborsClassifier(n_neighbors=1)

In [8]:
rf_model.fit(X_imputed, y)

KNeighborsClassifier(n_neighbors=1)

In [9]:
def predict_and_write_results(model, test_data_path, output_file_path):

    # Load test data
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.drop(columns=['Feature 21 (Discrete)','Feature 22 (Discrete)','Feature 23 (Discrete)','Feature 24'])
    test_data = pd.DataFrame(imputer.transform(test_data), columns=test_data.columns)
    test_data = lda.transform(test_data)

    # Make predictions on the test data
    predictions = model.predict(test_data)

    # Create a DataFrame for results with ID and Predicted Category columns
    results_df = pd.DataFrame({
        'ID': np.arange(1, len(predictions) + 1),  # Assuming IDs start from 1
        'Category': predictions
    })

    # Write the results to a CSV file
    results_df.to_csv(output_file_path, index=False)

# Example usage:
# Replace 'your_test_data.csv', 'output_results.csv' with your actual file paths
predict_and_write_results(rf_model, 'data/iith_foml_2023_test.csv', 'output_results.csv')


In [10]:
import pandas as pd

# Read the first CSV file (predictions from the first run)
df1 = pd.read_csv('output_results.csv')

# Read the second CSV file (predictions from the second run)
df2 = pd.read_csv('output_results65472.csv')

df2.rename(columns = {'Category':'prev'}, inplace = True)
df1.rename(columns = {'Category':'curr'}, inplace = True)

# Merge the two DataFrames on the index (assuming the rows are in the same order)
merged_df = pd.concat([df2, df1['curr']], axis=1)

# Create a new column 'Prediction_Difference' to store the differences between predictions
merged_df['Prediction_Difference'] = merged_df['curr'] - merged_df['prev']

# Display rows where the predictions differ
differing_predictions = merged_df[merged_df['Prediction_Difference'] != 0]
(differing_predictions)

,ID,prev,curr,Prediction_Difference
